In [24]:
# src/train_models.py

import os
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split


In [25]:


# 📂 Chargement du fichier complet
df_full = pd.read_csv("../data/processed/train_clean.csv") 

# 🎯 Séparation features / target
X = df_full.drop(columns=["TARGET"])
y = df_full["TARGET"]

# ✂️ Split 80/20 (stratifié)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 💾 Sauvegarde
df_train_split = X_train.copy()
df_train_split["TARGET"] = y_train
df_test_split = X_test.copy()
df_test_split["TARGET"] = y_test

df_train_split.to_csv("../data/processed/train_split.csv", index=False)
df_test_split.to_csv("../data/processed/test_split.csv", index=False)


In [26]:
# Chargement des données depuis preprocessing sauvegardé
print("📅 Chargement des données depuis train_clean.csv...")
data_path = Path.cwd() / "data" / "processed"
if not data_path.exists():
    data_path = Path.cwd().parent / "data" / "processed"
df = pd.read_csv(data_path / "train_split.csv")

X = df.drop(columns=["TARGET", "SK_ID_CURR"], errors='ignore')
y = df["TARGET"]


📅 Chargement des données depuis train_clean.csv...


In [28]:
# Conversion booléens
def bool_to_int(df):
    df = df.copy()
    for col in df.select_dtypes(include='bool'):
        df[col] = df[col].astype(int)
    return df

def bool_to_str(df):
    df = df.copy()
    for col in df.select_dtypes(include='bool'):
        df[col] = df[col].astype(str)
    return df



In [29]:
# Imputation simple pour SMOTE
def fill_na_for_smote(X):
    X = X.copy()
    for col in X.select_dtypes(include=["number"]):
        X[col] = X[col].fillna(X[col].mean())
    for col in X.select_dtypes(include=["object", "category"]):
        X[col] = X[col].fillna(X[col].mode()[0])
    return X



In [30]:
# Score métier

def business_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1][0]
    fp = cm[0][1]
    return 10 * fn + fp

scorer = make_scorer(business_score, greater_is_better=False)


In [31]:

# Préprocesseur

def get_preprocessor(X):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    return ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [33]:
# Suppression outliers

def remove_outliers_iqr(X, y):
    X_numeric = X.select_dtypes(include=["number"])
    Q1 = X_numeric.quantile(0.25)
    Q3 = X_numeric.quantile(0.75)
    IQR = Q3 - Q1
    mask = ~((X_numeric < (Q1 - 1.5 * IQR)) | (X_numeric > (Q3 + 1.5 * IQR))).any(axis=1)
    return X[mask], y[mask]



In [34]:
# Modèles
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier()
}



In [35]:
# Prétraitements
preprocessings = {
    "pipeline": lambda X, y: (get_preprocessor(bool_to_str(X)), bool_to_str(X), y),
    "smote": lambda X, y: (
        get_preprocessor(bool_to_int(X)),
        *SMOTE(random_state=42).fit_resample(bool_to_int(fill_na_for_smote(X)), y)
    ),
    "custom": lambda X, y: (get_preprocessor(bool_to_str(X)), *remove_outliers_iqr(bool_to_str(X), y))
}


In [37]:
#Création dossier
os.makedirs("models", exist_ok=True)

# Entraînement
for preproc_name, preproc_fn in preprocessings.items():
    for model_name, model in models.items():
        print(f"\n🚀 Entraînement: {model_name} + {preproc_name}")
        preprocessor, X_prep, y_prep = preproc_fn(X, y)

        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('clf', model)
        ])

        param_grid = {
            'clf__n_estimators': [50, 100],
            'clf__max_depth': [5, 10]
        } if hasattr(model, 'n_estimators') else {}

        grid = GridSearchCV(pipeline, param_grid=param_grid, scoring=scorer, cv=3)

        with mlflow.start_run(run_name=f"{model_name}_{preproc_name}"):
            grid.fit(X_prep, y_prep)

            best_model = grid.best_estimator_
            cost = -grid.best_score_
            auc = roc_auc_score(y_prep, best_model.predict_proba(X_prep)[:, 1])

            mlflow.log_params(grid.best_params_)
            mlflow.log_metric("business_cost", cost)
            mlflow.log_metric("auc", auc)
            mlflow.sklearn.log_model(best_model, f"{model_name}_{preproc_name}")

            filename = f"models/{model_name}_{preproc_name}.pkl"
            joblib.dump(best_model, filename)
            print(f"✅ Modèle enregistré: {filename} | Score métier: {cost:.2f} | AUC: {auc:.4f}")



🚀 Entraînement: RandomForest + pipeline


2025/04/30 17:58:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Modèle enregistré: models/RandomForest_pipeline.pkl | Score métier: 66200.00 | AUC: 0.7220

🚀 Entraînement: XGBoost + pipeline


/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:59:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_en

✅ Modèle enregistré: models/XGBoost_pipeline.pkl | Score métier: 63363.67 | AUC: 0.9862

🚀 Entraînement: LightGBM + pipeline
[LightGBM] [Info] Number of positive: 13240, number of negative: 150763
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12850
[LightGBM] [Info] Number of data points in the train set: 164003, number of used features: 361
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080730 -> initscore=-2.432467
[LightGBM] [Info] Start training from score -2.432467
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

...

[LightGBM] [Info] Number of positive: 1127, number of negative: 12441
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10924
[LightGBM] [Info] Number of data points in the train set: 13568, number of used features: 295
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083063 -> initscore=-2.401438
[LightGBM] [Info] Start training from score -2.401438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1690, number of negative: 18661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11060
[LightGBM] [Info] Number of data points in the train set: 20351, number of used features: 301
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083043 -> initscore=-2.401707
[LightGBM] [Info] Start training from score -2.401707


/opt/anaconda3/envs/ocp7_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/04/30 18:25:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Modèle enregistré: models/LightGBM_custom.pkl | Score métier: 5492.67 | AUC: 0.9654
